In [1]:
pip install -U transformers --upgrade

In [2]:
# !pip install -q pypdf
# !pip install -q python-dotenv
# !pip install  llama-index==0.10.12
# !pip install -q gradio
# !pip install einops
# !pip install accelerate

!pip install -q pypdf
!pip install -q python-dotenv
!pip install  llama-index
!pip install -q gradio
!pip install einops
!pip install accelerate

In [3]:
!pip install llama-index-llms-huggingface

In [4]:
!pip install llama-index-embeddings-fastembed

In [5]:
!pip install fastembed

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# import logging
# import sys

# logging.basicConfig(stream=sys.stdout, level=logging.INFO)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
# from llama_index.llms.huggingface import HuggingFaceLLM
# from llama_index.core import Settings

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext,PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings

documents = SimpleDirectoryReader("/content/drive/MyDrive/Learning/Data/").load_data()


In [8]:
from llama_index.embeddings.fastembed import FastEmbedEmbedding

embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.embed_model = embed_model
Settings.chunk_size = 512

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
from llama_index.core import PromptTemplate


system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."


# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")




In [10]:
from huggingface_hub import  notebook_login
notebook_login()

After getting the credentials from HuggingFace, its needed to send an access to request in order to use the gemma model. Follow the link to do so: https://huggingface.co/google/gemma-7b-it

In [11]:
import torch

llm = HuggingFaceLLM(
    context_window=8192,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="google/gemma-7b-it",
    model_name="google/gemma-7b-it",
    device_map="auto",
    #stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}
)

Settings.llm = llm
Settings.chunk_size = 512

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [12]:
index = VectorStoreIndex.from_documents(documents)


In [13]:
query_engine = index.as_query_engine()

def predict(input, history):
  response = query_engine.query(input)
  return str(response)

In [14]:

import gradio as gr

gr.ChatInterface(predict).launch(share=True)

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:334: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0d91924d6e9a2835d7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
